In [52]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import os
import pandas as pd


In [53]:
def setup(url):
    driver = webdriver.Chrome()
    driver.get(url)
    return driver

def teardown(driver):
    driver.quit()

def save_data_to_file(data, filename):
    if not os.path.exists('data'):
        os.makedirs('data')
    
    with open(f'data/{filename}', 'w', encoding='utf-8') as file:
        file.write(data)

In [57]:
url=[]
def scraper():
    driver = setup("https://community.shopify.com/c/shopify-apps/bd-p/shopify-apps")
    
    for page_no in range(1, 7):
        if page_no == 1:
            data = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.res-data")))
        else:
            pagination = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.pagination-div>a")))
            print(pagination[-1].get_attribute("outerHTML"))
            driver.implicitly_wait(2)
            pagination[-1].click() 
            data = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.res-data")))
        
        for i, ele in enumerate(data):
            try:
                data = driver.find_elements(By.CSS_SELECTOR, "div.res-data")
                divs=data[i]                
                link=divs.find_element(By.CSS_SELECTOR,"div.res-subject > a")
                post_url = link.get_attribute("href")
                url.append(post_url)
            
            
            except Exception as e:
                print(f"error occur : {e}")
    
    print(len(url))    
    teardown(driver)

scraper()

<a data-count="2" class="page" href="javascript:void(0)">Next</a>
<a data-count="3" class="page" href="javascript:void(0)">Next</a>
error occur : Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=128.0.6613.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7BD56B5D2+29090]
	(No symbol) [0x00007FF7BD4DE689]
	(No symbol) [0x00007FF7BD39B1CA]
	(No symbol) [0x00007FF7BD3A1CD3]
	(No symbol) [0x00007FF7BD3A4071]
	(No symbol) [0x00007FF7BD3A4110]
	(No symbol) [0x00007FF7BD3EEB4C]
	(No symbol) [0x00007FF7BD3EF22C]
	(No symbol) [0x00007FF7BD3E27CC]
	(No symbol) [0x00007FF7BD41672F]
	(No symbol) [0x00007FF7BD3E26A6]
	(No symbol) [0x00007FF7BD416900]
	(No symbol) [0x00007FF7BD4365D9]
	(No symbol) [0x00007FF7BD416493]
	(No symbol) [0x00007FF7BD3E09B1]
	(No symbol) [0x00007FF7BD3E1B11]
	GetHandleVer

In [58]:

def data_pages():
    for i,link in enumerate(url):
        driver = setup(link)
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.lia-quilt-row-main")))
        post_data = element.get_attribute("outerHTML")        
        save_data_to_file(post_data, f'page_{i+1}.html')
        teardown(driver)
        
data_pages()

In [59]:
d={"url":url,"title":[],"user_name":[],"user_title":[],"user_profile_link":[],"number_of_posts_by_user":[],"number_of_solutions_by_user":[],"number_of_kudos_by_user":[],"timestamp":[],"post_content":[],"solved":[],"labels_list":[],"number_of_views":[],"number_of_likes":[],"replies":[],"accepted_sol":[]}

for i,file in enumerate(os.listdir("data")):
    with open(f"data/{file}",encoding="utf8") as f:
# with open("data/page_105.html",encoding="utf8") as f:
        html_doc=f.read()
    soup=BeautifulSoup(html_doc,"html.parser")    
    
    title=soup.find("h1",class_="page-title")
    if title==None:
        continue
    d["title"].append(title.text)
    
    user_name=soup.find("a",class_="lia-user-name-link")
    d["user_name"].append(user_name.text)
    
    user_profile_link=user_name.get_attribute_list("href")
    d["user_profile_link"].append(user_profile_link[0])
   
    user_title_container=soup.find("span",class_="lia-message-unread")
    user_title=user_title_container.find_next("div")    
    d["user_title"].append(user_title.text.strip())
    
    number_of_posts_by_user=soup.find(class_="lia-message-author-posts-count")
    d["number_of_posts_by_user"].append(number_of_posts_by_user.text)
    
    number_of_solutions_by_user=soup.find(class_="lia-message-author-solutions-count")
    d["number_of_solutions_by_user"].append(number_of_solutions_by_user.text.strip())
    
    nunber_of_kudos_by_user=soup.find(class_="lia-message-author-kudos-count")
    d["number_of_kudos_by_user"].append(nunber_of_kudos_by_user.text.strip())
    
    publish_date=soup.select(".DateTime > meta")[0].get_attribute_list("content")
    d["timestamp"].append(publish_date[0])
    
    post_content_data=soup.find(class_="lia-message-body-content")
    for content in post_content_data(['style', 'script']):
        content.decompose().strip()
    post_content=' '.join(post_content_data.stripped_strings)
    d["post_content"].append(post_content)    
    
    if soup.find(class_="lia-panel-feedback-banner-safe"):
        solve="Y"
        accepted_sol=soup.find_all(class_="lia-message-body-content")
        for content in accepted_sol[1](['style', 'script']):
                content.decompose().strip()
        sol_content=' '.join(accepted_sol[1].stripped_strings)

    else:
        solve="N"
        sol_content=" "
    d["solved"].append(solve)
    d["accepted_sol"].append(sol_content)
    
    label_data=soup.find_all(class_="label")
    labels=[]
    for label_arr in label_data:
        labels.append((label_arr.text).strip())
    d["labels_list"].append(labels)
        
    view_count=soup.find(class_="lia-message-stats-count")
    if view_count:
        d["number_of_views"].append(view_count.text.strip())   
    else:
        d["number_of_views"].append("0")   
  
    
    like_count=soup.find(class_="MessageKudosCount")
    d["number_of_likes"].append(like_count.text.strip())  
    
    replies=soup.find(class_="lia-threaded-detail-display-message-view")
    reply_list=[]
    if replies:
        reply_data=soup.find_all(class_="lia-message-body-content")
        for content in reply_data[1](['style', 'script']):
            content.decompose().strip()
        reply_content=' '.join(reply_data[1].stripped_strings)
        reply_list.append(reply_content)
    else:
        pass
    d["replies"].append(reply_list)
    
    
  
    
df=pd.DataFrame(data=d);
df.to_csv("data.csv")    